In [2]:
from google.cloud import bigquery
from google.colab import auth

In [9]:
import pandas as pd

In [3]:
auth.authenticate_user()
print('Authenticated')

project_id = "jason-366210"
client = bigquery.Client(project=project_id)

Authenticated


# Problem Statement

Definisikan problem statement berdasarkan penjabaran masalah di bawah yang akan teman-teman jawab menggunakan SQL!

## Our Problem Statement

When and where do Austin City need to add more police to reduce the average time to clear a case with the most unsolved cases type?

# Penjabaran Masalah

## Masalah 1: Data preview

Tampilkan tabel crime sampai 5 baris pertama

In [5]:
client.query('''SELECT *
                FROM `bigquery-public-data.austin_crime.crime`
                LIMIT 5''') \
      .to_dataframe()

,unique_key,address,census_tract,clearance_date,clearance_status,council_district_code,description,district,latitude,longitude,location,location_description,primary_type,timestamp,x_coordinate,y_coordinate,year,zipcode
0,20165002898,"Austin, TX",None,2016-03-14 12:00:00+00:00,Not cleared,None,RAPE,UK,None,None,None,nan,Rape,2016-01-19 12:00:00+00:00,None,None,2016,None
1,20165004843,"Austin, TX",None,2016-03-31 12:00:00+00:00,Not cleared,None,RAPE,UK,None,None,None,nan,Rape,2016-01-25 12:00:00+00:00,None,None,2016,None
2,2016250988,"Austin, TX",None,2016-09-16 12:00:00+00:00,Not cleared,None,RAPE,UK,None,None,None,nan,Rape,2016-01-25 12:00:00+00:00,None,None,2016,None
3,20165003839,"Austin, TX",None,2016-02-09 12:00:00+00:00,Not cleared,None,AGG RAPE OF A CHILD,UK,None,None,None,nan,Rape,2016-01-27 12:00:00+00:00,None,None,2016,None
4,2016321746,"Austin, TX",None,2016-03-22 12:00:00+00:00,Not cleared,None,RAPE OF A CHILD,UK,None,None,None,nan,Rape,2016-02-01 12:00:00+00:00,None,None,2016,None


### Data Loading

In [10]:
pd.read_csv("./P0LC3_Jason Rich Darmawan Onggo Putra_1.csv")

,unique_key,address,census_tract,clearance_date,clearance_status,council_district_code,description,district,latitude,longitude,location,location_description,primary_type,timestamp,x_coordinate,y_coordinate,year,zipcode
0,20165002898,"Austin, TX",NaN,2016-03-14 12:00:00.000000 UTC,Not cleared,NaN,RAPE,UK,NaN,NaN,NaN,NaN,Rape,2016-01-19 12:00:00.000000 UTC,NaN,NaN,2016,NaN
1,20165004843,"Austin, TX",NaN,2016-03-31 12:00:00.000000 UTC,Not cleared,NaN,RAPE,UK,NaN,NaN,NaN,NaN,Rape,2016-01-25 12:00:00.000000 UTC,NaN,NaN,2016,NaN
2,2016250988,"Austin, TX",NaN,2016-09-16 12:00:00.000000 UTC,Not cleared,NaN,RAPE,UK,NaN,NaN,NaN,NaN,Rape,2016-01-25 12:00:00.000000 UTC,NaN,NaN,2016,NaN
3,20165003839,"Austin, TX",NaN,2016-02-09 12:00:00.000000 UTC,Not cleared,NaN,AGG RAPE OF A CHILD,UK,NaN,NaN,NaN,NaN,Rape,2016-01-27 12:00:00.000000 UTC,NaN,NaN,2016,NaN
4,2016321746,"Austin, TX",NaN,2016-03-22 12:00:00.000000 UTC,Not cleared,NaN,RAPE OF A CHILD,UK,NaN,NaN,NaN,NaN,Rape,2016-02-01 12:00:00.000000 UTC,NaN,NaN,2016,NaN


## Masalah 2: Jenis Tindak Kejahatan

Apa saja jenis tindak kejahatan yang terjadi di kota Austin selama tahun 2016?

### The Insights:

1. There are 6 primary type of cases.

ALERT: There are multiple Theft primary type. We will group it on later query.

In [131]:
client.query('''SELECT (CASE
                         WHEN primary_type LIKE '%Theft%' THEN 'Theft'
                         WHEN primary_type IN ('Agg Assault', 'Aggravated Assault') THEN 'Assault'
                         WHEN primary_type LIKE 'Burglary%' THEN 'Burglary'
                         WHEN primary_type LIKE '%Murder%' THEN 'Murder'
                         ELSE primary_type
                        END) AS new_primary_type
                FROM (SELECT DISTINCT primary_type
                      FROM `bigquery-public-data.austin_crime.crime`)
                GROUP BY new_primary_type''') \
      .to_dataframe()

,new_primary_type
0,Rape
1,Theft
2,Robbery
3,Assault
4,Burglary
5,Murder


### Data Loading

In [11]:
pd.read_csv("./P0LC3_Jason Rich Darmawan Onggo Putra_2.csv")

,new_primary_type
0,Rape
1,Theft
2,Robbery
3,Assault
4,Burglary
5,Murder


## Masalah 3: Clearance Status

Berapa banyak kasus tindak kejahatan berdasarkan kasus pembersihannya (clearance status) selama tahun 2016?

ALERT: clearance_status column have NaN rows.

### The Insights

1. cases with clearance status Not cleared is the most cases.
2. cases with clearance status Cleared by Arrest is the least cases.

In [94]:
client.query('''SELECT *, 
                       COUNT(clearance_status) AS count_clearance_status
                FROM (SELECT IFNULL(clearance_status, 'Not cleared') AS clearance_status,
                      FROM `bigquery-public-data.austin_crime.crime`)
                GROUP BY clearance_status''') \
      .to_dataframe()

,clearance_status,count_clearance_status
0,Not cleared,97300
1,Cleared by Exception,3087
2,Cleared by Arrest,16288


### Data Loading

In [12]:
pd.read_csv("./P0LC3_Jason Rich Darmawan Onggo Putra_3.csv")

,clearance_status,count_clearance_status
0,Not cleared,97300
1,Cleared by Exception,3087
2,Cleared by Arrest,16288


## Masalah 4: Jenis kejahatan yang tidak terselesaikan adalah Theft

Jenis kejahatan apa saja yang paling banyak belum terselesaikan kasusnya di tahun 2016?

### The insights

1. Theft is the most case with clearance status Not cleared.
2. Assault is the least case with clearance status Not cleared.

ALERT: There are multiple Theft primary type. Therefore, we will group it.

In [124]:
# ungrouped version, see next Code section
client.query('''SELECT primary_type, 
                       COUNT(primary_type) AS count_not_cleared
                FROM `bigquery-public-data.austin_crime.crime`
                WHERE clearance_status='Not cleared'
                 OR clearance_status IS NULL
                GROUP BY primary_type
                ORDER BY count_not_cleared DESC''') \
      .to_dataframe()

,primary_type,count_not_cleared
0,Theft,47843
1,Theft: All Other Larceny,11944
2,Theft: BOV,10273
3,Burglary,8952
4,Burglary / \nBreaking & Entering,5062
5,Auto Theft,5042
6,Robbery,1953
7,Theft: Shoplifting,1842
8,Agg Assault,1778
9,Rape,912


In [132]:
client.query('''SELECT new_primary_type, 
                       COUNT(new_primary_type) AS count_new_primary_type
                FROM (SELECT *, 
                             (CASE
                               WHEN primary_type LIKE '%Theft%' THEN 'Theft'
                               WHEN primary_type IN ('Agg Assault', 'Aggravated Assault') THEN 'Assault'
                               WHEN primary_type LIKE 'Burglary%' THEN 'Burglary'
                               WHEN primary_type LIKE '%Murder%' THEN 'Murder'
                               ELSE primary_type
                              END) AS new_primary_type,
                      FROM `bigquery-public-data.austin_crime.crime`)
                WHERE clearance_status='Not cleared'
                 OR clearance_status IS NULL
                GROUP BY new_primary_type
                ORDER BY new_primary_type DESC''') \
      .to_dataframe()

,new_primary_type,count_new_primary_type
0,Theft,77938
1,Robbery,1953
2,Rape,912
3,Murder,26
4,Burglary,14014
5,Assault,2457


### Data Loading

In [13]:
pd.read_csv("./P0LC3_Jason Rich Darmawan Onggo Putra_4.csv")

,new_primary_type,count_new_primary_type
0,Theft,77938
1,Robbery,1953
2,Rape,912
3,Murder,26
4,Burglary,14014
5,Assault,2457


## Masalah 5: Tahun 2016, kasus pencurian theft paling banyak di bulan Maret.

Di bulan apa di tahun 2016 kasus pencuriian (theft) sangat banyak terjadi?

Note: `theft` value is not present on column `primary_type`. Possible values are: `Theft`, `Theft: Purse Snatching`, etc.

### The Insights

1. In 2016, month March is when theft happens the most.
2. In 2016, the number of theft cases between 2139 to 2456.
3. The highest increase number of theft shappened on March as welll.
4. The highest decrease number of theft cases happened on February.

In [133]:
# Note: ORDER BY count_primary_type_theft
# The month_to_month_diff is correct, it compares between current month against previous month.
client.query('''SELECT *,
                       CONCAT(FORMAT("%'.2f", 
                                             ((count_primary_type_theft 
                                               / LAG(count_primary_type_theft, 1) OVER (ORDER BY month))
                                              - 1)
                                             *100)
                              ,'%') AS month_to_month_diff
                FROM (SELECT EXTRACT(YEAR FROM timestamp) AS year,
                             EXTRACT(MONTH FROM timestamp) AS month,
                             COUNT(primary_type) AS count_primary_type_theft
                      FROM `bigquery-public-data.austin_crime.crime`
                      WHERE timestamp BETWEEN "2016-01-01" AND "2016-12-31"
                       AND primary_type LIKE "%Theft%"
                      GROUP BY year, month
                      ORDER BY count_primary_type_theft DESC)''') \
      .to_dataframe()

,year,month,count_primary_type_theft,month_to_month_diff
0,2016,1,2416,None
1,2016,11,2280,-4.96%
2,2016,4,2442,-0.57%
3,2016,7,2309,-5.21%
4,2016,8,2384,3.25%
5,2016,6,2436,3.88%
6,2016,3,2456,14.82%
7,2016,2,2139,-11.47%
8,2016,10,2399,2.78%
9,2016,12,2273,-0.31%


### Data Loading

In [14]:
pd.read_csv("./P0LC3_Jason Rich Darmawan Onggo Putra_5.csv")

,year,month,count_primary_type_theft,month_to_month_diff
0,2016,4,2442,-0.57%
1,2016,10,2399,2.78%
2,2016,12,2273,-0.31%
3,2016,3,2456,14.82%
4,2016,11,2280,-4.96%
5,2016,1,2416,NaN
6,2016,5,2345,-3.97%
7,2016,6,2436,3.88%
8,2016,2,2139,-11.47%
9,2016,8,2384,3.25%


## Masalah 6: Tahun 2016, distrik D paling banyak terjadi kasus pencurian

Di distrik apa yang paling banyak terjadi kasus pencurian selama 2016?

Note: `theft` value is not present on column `primary_type`. Possible values are: `Theft`, `Theft: Purse Snatching`, etc.

### The Insights

1. district D is where theft the most happened.
2. district 88 is where theft the least happened.

In [26]:
client.query('''SELECT district,
                       COUNT(district) AS count_primary_type_theft
                FROM `bigquery-public-data.austin_crime.crime`
                WHERE timestamp BETWEEN "2016-01-01" AND "2016-12-31"
                 AND primary_type LIKE "%Theft%"
                GROUP BY district
                ORDER BY count_primary_type_theft DESC''') \
      .to_dataframe()

,district,count_primary_type_theft
0,D,4401
1,B,3520
2,F,3302
3,E,3279
4,A,3271
5,I,3071
6,H,3036
7,G,2190
8,C,1883
9,AP,161


### Data Loading

In [15]:
pd.read_csv("./P0LC3_Jason Rich Darmawan Onggo Putra_6.csv")

,district,count_primary_type_theft
0,D,4401
1,B,3520
2,F,3302
3,E,3279
4,A,3271
5,I,3071
6,H,3036
7,G,2190
8,C,1883
9,AP,161


## Masalah 7: rata-rata 14 hari, waktu yang dibutuhkan untuk menyelesaikan kasus tindak kejahatan theft.

Berapa lama rata-rata waktu (dalam hari) kasus tindak kejahatan 'theft' terselesaikan sejak kasus tersebut terangkat selama tahun 2016? (Hint: Hitung selisih hari dari timestamp hingga clearance date)

Note: `theft` value is not present on column `primary_type`. Possible values are: `Theft`, `Theft: Purse Snatching`, etc.

In [50]:
client.query('''SELECT AVG(DATE_DIFF(clearance_date, timestamp, DAY)) AS date_diff_mean
                FROM `bigquery-public-data.austin_crime.crime`
                WHERE timestamp BETWEEN "2016-01-01" AND "2016-12-31"
                 AND primary_type LIKE "%Theft%"
                 AND clearance_status IN ("Cleared by Arrest", "Cleared by Exception")
                ''') \
      .to_dataframe()

,date_diff_mean
0,14.092904


### Data Loading

In [16]:
pd.read_csv("./P0LC3_Jason Rich Darmawan Onggo Putra_7.csv")

,date_diff_mean
0,14.092904


# Poin Analisis

Dari penjabaran yang sudah kamu bahas (pertanyaan 2-7), apa kesimpulan yang kamu dapat sampaikan?

## The conclusion

Theft is the most cases with clearance status Not cleared. By average, in 2016, it requires Austin city 14 days to clear a theft case. Also, on March is when theft most happened. If we dig deeper, district D is where theft most happened.